In [2]:
from pymongo import MongoClient
import pandas as pd
import numpy as np

# Connect to MongoDB
client = MongoClient("mongodb://localhost:27017/")
db = client["ais_training_data"]
collection = db["station"]

# Load and filter valid sequences
data = list(collection.find({
    "Visited_Ports": {"$exists": True, "$ne": []},
    "Ship_Type": {"$exists": True},
    "First_Port": {"$exists": True},
    "Last_Port": {"$exists": True}
}))

df = pd.DataFrame(data)
df = df.dropna(subset=["Ship_Type", "First_Port", "Last_Port", "Visited_Ports"])

valid_ship_types = np.array([
    'Sailing', 'Military', 'Tug', 'Fishing', 'Pilot',
    'Other', 'Port tender', 'Cargo', 'Pleasure', 'Passenger',
    'Reserved', 'Tanker', 'SAR', 'HSC', 'Dredging',
    'Not party to conflict', 'Law enforcement', 'Towing', 'Diving',
    'Anti-pollution', 'Medical', 'Spare 1', 'WIG', 'Towing long/wide',
    'Spare 2'
], dtype=object)

df["Ship_Type"] = df["Ship_Type"].apply(
    lambda x: np.random.choice(valid_ship_types) if x == "Undefined" else x
)


In [3]:
from sklearn.preprocessing import LabelEncoder

# Create vocabulary
all_ports = set(sum(df["Visited_Ports"].tolist(), []))
all_ports.update(df["First_Port"].tolist())
all_ports.update(df["Last_Port"].tolist())

port_encoder = LabelEncoder()
df["Visited_Ports_Str"] = df["Visited_Ports"].apply(lambda x: ','.join(x))
port_encoder.fit(list(all_ports))

ship_encoder = LabelEncoder()
df["Ship_Type"] = ship_encoder.fit_transform(df["Ship_Type"])

# Encode port sequences
df["Visited_Ports_Encoded"] = df["Visited_Ports"].apply(lambda ports: port_encoder.transform(ports).tolist())
df["First_Port_Encoded"] = port_encoder.transform(df["First_Port"])
df["Last_Port_Encoded"] = port_encoder.transform(df["Last_Port"])


In [4]:
from torch.utils.data import Dataset
import torch
from torch.nn.utils.rnn import pad_sequence

class RouteDataset(Dataset):
    def __init__(self, df):
        self.inputs = [
            torch.tensor([ship, first, last], dtype=torch.long)
            for ship, first, last in zip(df["Ship_Type"], df["First_Port_Encoded"], df["Last_Port_Encoded"])
        ]
        self.targets = [torch.tensor(seq, dtype=torch.long) for seq in df["Visited_Ports_Encoded"]]

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return self.inputs[idx], self.targets[idx]


In [5]:
import torch.nn as nn

class RouteLSTM(nn.Module):
    def __init__(self, port_vocab_size, ship_vocab_size, hidden_dim=128):
        super().__init__()
        self.port_emb = nn.Embedding(port_vocab_size, 32)
        self.ship_emb = nn.Embedding(ship_vocab_size, 8)
        self.lstm = nn.LSTM(input_size=72, hidden_size=hidden_dim, batch_first=True)

        self.fc = nn.Linear(hidden_dim, port_vocab_size)

    def forward(self, ship_type, first_port, last_port, target_len):
        # Create embeddings
        ship_embed = self.ship_emb(ship_type).unsqueeze(1).expand(-1, target_len, -1)  # [B, T, 8]
        first_embed = self.port_emb(first_port).unsqueeze(1).expand(-1, target_len, -1)
        last_embed = self.port_emb(last_port).unsqueeze(1).expand(-1, target_len, -1)

        combined = torch.cat([first_embed, last_embed, ship_embed], dim=2)
        out, _ = self.lstm(combined)
        out = self.fc(out)
        return out


In [16]:
from torch.utils.data import DataLoader
import torch.optim as optim

dataset = RouteDataset(df)
loader = DataLoader(dataset, batch_size=32, shuffle=True, collate_fn=lambda batch: (
    torch.stack([b[0] for b in batch]),
    pad_sequence([b[1] for b in batch], batch_first=True, padding_value=0)
))

model = RouteLSTM(port_vocab_size=len(port_encoder.classes_), ship_vocab_size=len(ship_encoder.classes_))
criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = optim.Adam(model.parameters(), lr=0.01)

EPOCHS = 100
model.train()
for epoch in range(EPOCHS):
    total_loss = 0
    for features, targets in loader:
        ship_type = features[:, 0]
        first_port = features[:, 1]
        last_port = features[:, 2]

        target_len = targets.shape[1]
        outputs = model(ship_type, first_port, last_port, target_len)

        loss = criterion(outputs.view(-1, outputs.shape[2]), targets.view(-1))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    
    print(f"Epoch {epoch+1}/{EPOCHS}, Loss: {total_loss:.4f}")


Epoch 1/100, Loss: 4052.6907
Epoch 2/100, Loss: 3773.9596
Epoch 3/100, Loss: 3514.3987
Epoch 4/100, Loss: 3272.6896
Epoch 5/100, Loss: 3047.6044
Epoch 6/100, Loss: 2837.9999
Epoch 7/100, Loss: 2642.8113
Epoch 8/100, Loss: 2461.0472
Epoch 9/100, Loss: 2291.7842
Epoch 10/100, Loss: 2134.1626
Epoch 11/100, Loss: 1987.3817
Epoch 12/100, Loss: 1850.6960
Epoch 13/100, Loss: 1723.4110
Epoch 14/100, Loss: 1604.8803
Epoch 15/100, Loss: 1494.5018
Epoch 16/100, Loss: 1391.7147
Epoch 17/100, Loss: 1295.9970
Epoch 18/100, Loss: 1206.8625
Epoch 19/100, Loss: 1123.8583
Epoch 20/100, Loss: 1046.5629
Epoch 21/100, Loss: 974.5837
Epoch 22/100, Loss: 907.5549
Epoch 23/100, Loss: 845.1362
Epoch 24/100, Loss: 787.0104
Epoch 25/100, Loss: 732.8824
Epoch 26/100, Loss: 682.4771
Epoch 27/100, Loss: 635.5385
Epoch 28/100, Loss: 591.8282
Epoch 29/100, Loss: 551.1241
Epoch 30/100, Loss: 513.2195
Epoch 31/100, Loss: 477.9219
Epoch 32/100, Loss: 445.0520
Epoch 33/100, Loss: 414.4427
Epoch 34/100, Loss: 385.9387
Epo

In [15]:
# Save model state
torch.save({
    "model_state_dict": model.state_dict(),
    "port_encoder": port_encoder,
    "ship_encoder": ship_encoder
}, "best_route_lstm_model.pth")


In [7]:
def predict_route(model, ship_type_str, first_port_str, last_port_str, max_len=10):
    model.eval()
    with torch.no_grad():
        ship_type = torch.tensor([ship_encoder.transform([ship_type_str])[0]])
        first_port = torch.tensor([port_encoder.transform([first_port_str])[0]])
        last_port = torch.tensor([port_encoder.transform([last_port_str])[0]])

        output = model(ship_type, first_port, last_port, target_len=max_len)
        pred_ids = torch.argmax(output, dim=2).squeeze(0).tolist()
        return [port_encoder.inverse_transform([i])[0] for i in pred_ids if i < len(port_encoder.classes_)]


In [13]:
def process(seq):
    seen = set()
    result = []
    for port in seq:
        if port not in seen:
            seen.add(port)
            result.append(port)
    return result


In [14]:
route = predict_route(model, "Cargo", "Aalborg", "Havdrup")
cleaned_route = process(route)
print("Predicted route:", cleaned_route)


Predicted route: ['Aalborg', 'Copenhagen', 'Grenaa', 'Samso Island', 'Kastrup', 'Kyndby']
